Import dependencies

In [2]:
import landmark_detector as ld
import os
import random
import numpy as np
import keras
from tqdm.notebook import tqdm
from keras import layers

Set variables

In [3]:
words = ['deaf', 'eat', 'fish', 'friend', 'like', 'milk', 'nice', 'no', 'orange', 'teacher', 'want', 'what', 'where', 'yes']
select_words = ['no', 'eat', 'teacher', 'want', 'fish']
modes = ['train', 'val', 'test']
path = '../preprocessing/dataset/'
num_features = 126
model_name = 'draft_model'
fps = 20

Get data from mediapipe


In [ ]:
def get_data(mode, words, path, detector_path):
    detector = ld.get_detector(detector_path)

    training_X = []
    training_y = []

    num_videos = 0
    highest_frame = 0

    bad_videos = 0

    for word in tqdm(words, desc=mode):
        word_path = os.path.join(path, mode, word)
        
        video_files = [f for f in os.listdir(word_path) if f.endswith('.mp4')]
        
        for video_file in tqdm(video_files, desc=word):
            video_path = os.path.join(word_path, video_file)
            
            try:
                video_X = []
                landmarks, current_frames = ld.get_landmarks(video_path, detector)
                
                if len(landmarks) == 0:
                    bad_videos+=1
                    continue
                
                if current_frames > highest_frame:
                    highest_frame = current_frames
                
                for frame in range(len(landmarks)):
                    features = np.array(landmarks[frame]).flatten()
                    video_X.append(features)
                
                training_X.append(video_X)
                training_y.append(words.index(word))
                num_videos += 1

            except Exception as e:
                print(f"Error processing video {video_file}: {e}")
                continue 

    return training_X, training_y, num_videos, highest_frame, bad_videos

training_X, training_y, num_videos, highest_frame, bad_videos = get_data('train', select_words, path, '../models/hand_landmarker.task')

print('Number of videos:', num_videos)
print('Highest frame:', highest_frame)
print('Videos with no landmarkers detected: ', bad_videos)


train:   0%|          | 0/5 [00:00<?, ?it/s]

no:   0%|          | 0/27 [00:00<?, ?it/s]

eat:   0%|          | 0/28 [00:00<?, ?it/s]

teacher:   0%|          | 0/27 [00:00<?, ?it/s]

want:   0%|          | 0/27 [00:00<?, ?it/s]

fish:   0%|          | 0/29 [00:00<?, ?it/s]

Number of videos: 138
Highest frame: 47
Videos with no landmarkers detected:  0


Padding and Masking X

In [5]:
def padX(X, num_videos, highest_frame, num_features):
    padded_X = np.zeros((num_videos, highest_frame, num_features))
    mask = np.ones((num_videos, highest_frame, num_features)) 
    for i in range(num_videos):
        video = X[i]
        for j in range(len(video)):
            frame = video[j]
            if len(frame) < num_features:
                padded_X[i, j, :] = np.pad(frame, (0, num_features - len(frame)), 'constant')
                mask[i, j, len(frame):] = 0
            else:
                padded_X[i, j, :] = frame
        if len(video) < highest_frame:
            mask[i, len(video):, :] = 0

    return padded_X, mask

padded_X, mask = padX(training_X, num_videos, highest_frame, num_features)
print(padded_X.shape)

(138, 47, 126)


Create model

In [6]:
model = keras.Sequential()

model.add(keras.Input(shape=(highest_frame, num_features)))
model.add(layers.Masking(mask_value=0.0))
model.add(layers.LSTM(64))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(select_words), activation='sigmoid'))


model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 47, 126)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,141 (199.77 KB)

 Trainable params: 51,141 (199.77 KB)

 Non-trainable params: 0 (0.00 B)

Train model

In [7]:
val_X, val_y, num_val_videos, highest_frame_val, bad_videos = get_data('val', select_words, path, '../models/hand_landmarker.task')
padded_val_X, val_mask = padX(val_X, num_val_videos, highest_frame, num_features)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_X, np.array(training_y), epochs=10, validation_data=(padded_val_X, np.array(val_y)))

model.save(f'../models/{model_name}.keras')

with open(f"../models/{model_name}.env", "w") as file:
    file.write(f"MAX_FRAMES={highest_frame}\n")
    file.write(f"NUM_FEATURES={num_features}\n")
    file.write(f"WORDS={",".join(select_words)}\n")
    file.write(f"FPS={fps}\n")

val:   0%|          | 0/5 [00:00<?, ?it/s]

no:   0%|          | 0/6 [00:00<?, ?it/s]

eat:   0%|          | 0/6 [00:00<?, ?it/s]

teacher:   0%|          | 0/6 [00:00<?, ?it/s]

want:   0%|          | 0/6 [00:00<?, ?it/s]

fish:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.3036 - loss: 1.5929 - val_accuracy: 0.3000 - val_loss: 1.5663
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3355 - loss: 1.5308 - val_accuracy: 0.3000 - val_loss: 1.5625
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.2617 - loss: 1.5583 - val_accuracy: 0.4000 - val_loss: 1.5297
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4642 - loss: 1.4077 - val_accuracy: 0.4333 - val_loss: 1.5175
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4613 - loss: 1.3886 - val_accuracy: 0.4000 - val_loss: 1.5019
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4636 - loss: 1.3729 - val_accuracy: 0.5333 - val_loss: 1.4850
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4842 - loss: 1.3442 - val_accuracy: 0.4667 - val_loss: 1.4767
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5420 - loss: 1.2791 - val_accuracy: 0.5333 - val_loss: 1.4540


Evaluate model

In [8]:
test_X, test_y, num_test_videos, highest_frame_test, bad_videos = get_data('test', select_words, path, '../models/hand_landmarker.task')
padded_test_X, test_mask = padX(test_X, num_test_videos, highest_frame, num_features)
results = model.evaluate(padded_test_X, np.array(test_y))

print('Test loss:', results)

test:   0%|          | 0/5 [00:00<?, ?it/s]

no:   0%|          | 0/5 [00:00<?, ?it/s]

eat:   0%|          | 0/6 [00:00<?, ?it/s]

teacher:   0%|          | 0/5 [00:00<?, ?it/s]

want:   0%|          | 0/5 [00:00<?, ?it/s]

fish:   0%|          | 0/6 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5556 - loss: 1.3493
Test loss: [1.3492510318756104, 0.5555555820465088]


Use model

In [9]:
detector = ld.get_detector('../models/hand_landmarker.task')

prediction_X = []
prediction_y = []

test_path = path + 'test/teacher/'
files = os.listdir(test_path)
mp4_files = [f for f in files if f.endswith('.mp4')]
random_file = random.choice(mp4_files)
video_path = os.path.join(test_path, random_file)

video_X = []
landmarks, frame_count = ld.get_landmarks(video_path, detector)
if len(landmarks) == 0:
    print('No landmarks detected')
else:
    for frame in range(len(landmarks)):
        features = np.array(landmarks[frame]).flatten()
        features = np.pad(features, (0, num_features - len(features)), 'constant')
        video_X.append(features)
    for i in range(highest_frame-len(video_X)):
        temp = np.zeros((num_features))
        video_X.append(temp)

    prediction_X.append(video_X)
    prediction_y.append(select_words.index('teacher'))


    print(np.shape(prediction_X))
    print(model.predict(np.array(prediction_X)))

(1, 47, 126)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
[[0.2288182  0.45364273 0.76885283 0.6003365  0.18212897]]
